<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2024/blob/main/DynamicDistillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from scipy.integrate import solve_ivp
import numpy as np
from plotly.subplots import make_subplots

In [19]:
#0  Condenser
#1
#2  Nr-1
#3  Feed Stage
#4
#5
#6  Nr+Ns-1
#7  Nr+Ns:  Reboiler

Nr = 3
Ns = 4

F=1.5  #mol/s
z=0.4
q = 0.7

alpha=2.
stage_holdup=0.5
feedstage_holdup = 0.75
condenser_holdup = 5.
reboiler_holdup = 5.
m_holdup=np.r_[condenser_holdup, np.full(Nr-1, stage_holdup), feedstage_holdup, np.full(Ns-1, stage_holdup) ,reboiler_holdup]
W=1.  # 1/(s mol**0.5)

#initial conditions
m0 = m_holdup*1.05  #initial number moles on each stage is set slightly higher than holdup to yield a little flow
x0 = np.full_like(m0, z)  #fill each stage with feed composition


In [24]:
def rhs(t, v):
    R=2.
    m,x = np.split(v,2)
    y=alpha*x/(1-x+alpha*x)
    dm = np.zeros_like(m)
    dmx = np.zeros_like(m)
    V = np.zeros_like(m)

    L =  np.where(m<m_holdup, 0., W*(m-m_holdup)**1.5)

    #On the condenser, there is weir, the overflowing liquid is split into D and L[0]
    D = L[0]/(R+1)
    L[0]-=D

    Vrec = L[0] + D

    V[1:Nr+1] = Vrec  #constant vapor flow rate in rectifying section
    V[Nr+1:] = Vrec - F*(1-q)  #constant vapor flow rate in stripping section

    dm[1:-1] =  L[:-2] - L[1:-1] + V[2:] - V[1:-1]
    #  1to6      0to5      1to6     2to7     1to6
    dm[Nr]+=F
    dm[0] = V[1] - L[0] - D
    dm[-1] = L[-2] - V[-1] - L[-1]

    dmx[1:-1] =  x[:-2]*L[:-2] - x[1:-1]*L[1:-1] + y[2:]*V[2:] - y[1:-1]*V[1:-1]
    dmx[Nr]+=z*F
    dmx[0] = y[1]*V[1] - x[0]*L[0] - x[0]*D
    dmx[-1] = x[-2]*L[-2] - y[-1]*V[-1] - x[-1]*L[-1]

    dx = (dmx - x*dm)/m
    return np.r_[dm, dx]

In [28]:
tend=1000
res=solve_ivp(rhs, (0, tend), np.r_[m0, x0], method='Radau', dense_output=True)
res

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  7.366e-02 ...  7.063e+02  1.000e+03]
        y: [[ 5.250e+00  5.250e+00 ...  5.250e+00  5.250e+00]
            [ 5.250e-01  5.308e-01 ...  6.908e-01  6.908e-01]
            ...
            [ 4.000e-01  4.000e-01 ...  3.421e-01  3.421e-01]
            [ 4.000e-01  4.008e-01 ...  3.912e-01  3.912e-01]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x7d5d499fa8c0>
 t_events: None
 y_events: None
     nfev: 163
     njev: 6
      nlu: 40

In [29]:
tplot=np.linspace(0,tend,9)

In [30]:
res.sol(tplot).shape

(16, 9)